# Anomalia de campo total produzida por um prisma

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html).

### Coisas para importar

In [1]:
#%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

### Pontos onde são calculados os dados

In [2]:
shape = (100, 100) # número de pontos ao longo de x e y
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

xc = 0.5*(area[1] + area[0])
yc = 0.5*(area[3] + area[2])

### Inclinação e a declinação do campo geomagnético local

In [3]:
inc = -30. # graus
dec = 0. # graus

In [4]:
amp = 23500.0 + 0.05*(xp - xc) + 0.08*(yp - yc) # nT

In [5]:
F = utils.ang2vec(1.0, inc, dec) # versor do campo geomagnético local

In [6]:
print F

[ 0.8660254  0.        -0.5      ]


### Modelo de prisma

In [7]:
mpl.close('all')
axis = mpl.figure().gca()
mpl.axis('scaled')
vertices = mpl.draw_polygon(area, axis, xy2ne=True)

In [8]:
print vertices

[[ 3059.62343096 -3321.12970711]
 [ 3268.82845188 -2275.10460251]
 [ 3530.33472803  -810.66945607]
 [ 3347.28033473   366.10878661]
 [ 3059.62343096  1516.73640167]
 [ 2248.9539749   2144.35146444]
 [  993.72384937  2824.26778243]
 [ -209.20502092  3347.28033473]
 [-1804.39330544  3634.93723849]
 [-3425.73221757  3739.53974895]
 [-4027.19665272  3294.9790795 ]
 [-4367.15481172  2275.10460251]
 [-4288.70292887   758.36820084]
 [-3974.89539749  -575.31380753]
 [-3451.88284519 -1490.58577406]
 [-2641.21338912 -1725.94142259]
 [-2510.46025105 -1176.77824268]
 [-2876.56903766  -444.56066946]
 [-3399.58158996   679.91631799]
 [-3373.43096234  2458.15899582]
 [-2562.76150628  2928.87029289]
 [-1569.0376569   2745.81589958]
 [ -339.958159    2144.35146444]
 [ 1255.23012552  1333.68200837]
 [ 1778.24267782   366.10878661]
 [ 1882.84518828 -1281.38075314]
 [ 1647.48953975 -3190.37656904]
 [ 1856.69456067 -4131.79916318]
 [ 2562.76150628 -4419.45606695]
 [ 2928.87029289 -3948.74476987]]


In [9]:
incs = inc
decs = dec
m = 3.

model = [mesher.PolygonalPrism(vertices, 10, 1010, {'magnetization': utils.ang2vec(m, incs, decs)})]

### Anomalia de campo total (em nT)

In [10]:
bx = polyprism.bx(xp, yp, zp, model)
by = polyprism.by(xp, yp, zp, model)
bz = polyprism.bz(xp, yp, zp, model)

In [11]:
act_aprox = polyprism.tf(xp, yp, zp, model, inc, dec)

In [12]:
ct = np.sqrt((amp*F[0] + bx)**2 + (amp*F[1] + by)**2 + (amp*F[2] + bz)**2)

In [13]:
act = ct - amp

### Plot da anomalia de campo total

In [14]:
mpl.close('all')
mpl.figure(figsize=(12,10))

mpl.subplot(2,2,1)
mpl.title("Campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, ct, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,2)
mpl.title("Anomalia de campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,3)
mpl.title("Anomalia de campo total aproximada (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act_aprox, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,4)
mpl.title("Diferenca act_aprox - act (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act_aprox - act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout(pad = 2.)

mpl.savefig('prisma_mag.png', dpi=300)

#mpl.show()

<img src='prisma_mag.png', width=1200>